source: https://www.kaggle.com/aakashns/pytorch-basics-linear-regression-from-scratch

In [1]:
# Import Numpy & PyTorch
import numpy as np
import torch

In [2]:
# Create tensors.
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)

In [3]:
# Print tensors
print(x)
print(w)
print(b)

tensor(3.)
tensor(4., requires_grad=True)
tensor(5., requires_grad=True)


In [4]:
# Arithmetic operations
y = w * x + b
print(y)

tensor(17., grad_fn=<AddBackward0>)


## What makes PyTorch special, is that we can automatically compute the derivative of y w.r.t. the tensors that have requires_grad set to True i.e. w and b.

In [5]:
# Compute gradients
y.backward()

In [6]:
# Display gradients
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dw: tensor(3.)
dy/db: tensor(1.)


## Problem Statement
We'll create a model that predicts crop yeilds for apples and oranges (target variables) by looking at the average temperature, rainfall and humidity (input variables or features) in a region. Here's the training data:

<img src=".\reg_imagen.png">

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

yeild_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1 

yeild_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

Visually, it means that the yield of apples is a linear or planar function of the temperature, rainfall & humidity.

<img src=".\reg_graf.png">

Our objective: Find a suitable set of weights and biases using the training data, to make accurate predictions.

## Training Data

The training data can be represented using 2 matrices (inputs and targets), each with one row per observation and one column per variable.

In [7]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [8]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

Before we build a model, we need to convert inputs and targets to PyTorch tensors.

In [9]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear Regression Model (from scratch)
The weights and biases can also be represented as matrices, initialized with random values. The first row of w and the first element of b are use to predict the first target variable i.e. yield for apples, and similarly the second for oranges.

In [10]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.5176,  0.4022, -1.4352],
        [ 1.3922, -1.7749,  0.5079]], requires_grad=True)
tensor([-0.3327,  0.5732], requires_grad=True)


The model is simply a function that performs a matrix multiplication of the input x and the weights w (transposed) and adds the bias b (replicated for each observation).

<img src=".\reg_matrix.png">

In [11]:
# Define the model
def model(x):
    return x @ w.t() + b

In [12]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  2.6874,   5.1324],
        [ -9.6881,   3.5871],
        [ 15.3549, -86.6726],
        [ 16.6552,  85.0567],
        [-26.4683, -38.1937]], grad_fn=<AddBackward0>)


In [13]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Because we've started with random weights and biases, the model does not a very good job of predicting the target varaibles.

## Loss Function
We can compare the predictions with the actual targets, using the following method:

- Calculate the difference between the two matrices (preds and targets).
- Square all elements of the difference matrix to remove negative values.
- Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the mean squared error (MSE).

In [14]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [15]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(12757.1895, grad_fn=<DivBackward0>)


## Compute Gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have requires_grad set to True.

In [16]:
# Compute gradients
loss.backward()

The gradients are stored in the .grad property of the respective tensors.

In [17]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.5176,  0.4022, -1.4352],
        [ 1.3922, -1.7749,  0.5079]], requires_grad=True)
tensor([[ -6128.0088,  -7619.9443,  -4673.6870],
        [ -7731.2007, -11075.7490,  -6198.0400]])


Finally, we'll reset the gradients to zero before moving forward, because PyTorch accumulates gradients.

In [18]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Adjust weights and biases using gradient descent
We'll reduce the loss and improve our model using the gradient descent algorithm, which has the following steps:

1. Generate predictions
2. Calculate the loss
3. Compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient
5. Reset the gradients to zero

In [19]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  2.6874,   5.1324],
        [ -9.6881,   3.5871],
        [ 15.3549, -86.6726],
        [ 16.6552,  85.0567],
        [-26.4683, -38.1937]], grad_fn=<AddBackward0>)


In [20]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(12757.1895, grad_fn=<DivBackward0>)


In [21]:
# Compute gradients
loss.backward()

In [22]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

With the new weights and biases, the model should have a lower loss.

In [23]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(9673.2363, grad_fn=<DivBackward0>)


## Train for multiple epochs
To reduce the loss further, we repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch.

In [24]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [25]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(1050.5520, grad_fn=<DivBackward0>)


In [26]:
# Print predictions
#print(preds)
#print(targets)
print("Apples: ")
for i, j in zip(preds,targets):
    print("Prediction: ", i.detach().numpy()[0],"Target: ", j.detach().numpy()[0])
print("\n")    
print("Oranges: ")
for i, j in zip(preds,targets):
    print("Prediction: ", i.detach().numpy()[1],"Target: ", j.detach().numpy()[1])

Apples: 
Prediction:  63.31107 Target:  56.0
Prediction:  73.68128 Target:  81.0
Prediction:  128.20143 Target:  119.0
Prediction:  55.846672 Target:  22.0
Prediction:  66.73578 Target:  103.0


Oranges: 
Prediction:  82.65945 Target:  70.0
Prediction:  111.35926 Target:  101.0
Prediction:  88.79476 Target:  133.0
Prediction:  108.47761 Target:  37.0
Prediction:  96.14348 Target:  119.0
